# Pseudo Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_pseudodata.ipynb)

### Training

In [1]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

In [9]:
df = pd.read_csv('../datasets/pseudodata_train.csv')
x_train = df.drop('E', axis=1)
y_train = df['E']

In [10]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [11]:
model = iblm.fit(x_train, y_train, model_name = 'psedodata', file_path='./model_code/')

> Start of model creating.
Tokens Used: 4543
	Prompt Tokens: 4265
	Completion Tokens: 278
Successful Requests: 1
Total Cost (USD): $0.14462999999999998


In [12]:
# Code of the model created
print(model)

import numpy as np
import pandas as pd

def predict(x):
    df = x.copy()

    output = []
    for index, row in df.iterrows():

        # Feature creation and data preprocessing
        A, B, C, D = row['A'], row['B'], row['C'], row['D']

        # Custom logic for binary classification
        y = A * C - B * D

        # Apply sigmoid function to convert y to probability
        y = 1 / (1 + np.exp(-y))
        output.append(y)

    output = np.array(output)
        
    return output

# Example usage:
data = pd.DataFrame({
    'A': [-0.360007291614975, -1.4266828135942111, -0.8713033349256126],
    'B': [-0.3006233204725373, 0.0063185223230947, -0.2553967691046434],
    'C': [1.781525021076077, -1.5914413231764364, 0.9008405013518856],
    'D': [-1.373444579582277, -1.528047075881549, -1.7806373818932837]
})

predictions = predict(data)
print(predictions)


### Prediction

In [13]:
df = pd.read_csv('../datasets/pseudodata_test.csv')
x_test = df.drop('E', axis=1)
y_test = df['E']

In [14]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

[0.25840954 0.90721858 0.22449267]


In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.415
Precision: 0.43103448275862066
Recall: 0.49504950495049505
F1 score: 0.4608294930875576
ROC-AUC: 0.35973597359735976


### Prediction from external files


In [ ]:
from .model_code import pseudodata

y_proba = pseudodata.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.415
Precision: 0.43103448275862066
Recall: 0.49504950495049505
F1 score: 0.4608294930875576
ROC-AUC: 0.35973597359735976


### Interpretation of results

In [57]:
description = iblm.interpret()

Tokens Used: 881
	Prompt Tokens: 537
	Completion Tokens: 344
Successful Requests: 1
Total Cost (USD): $0.036750000000000005


In [58]:
print(description)

- Data preprocessing:
    - Fill missing 'age' values with the median age.
    - Fill missing 'fare' values with the median fare.
    - Fill missing 'embarked' values with the mode (most frequent) of the 'embarked' column.

- Feature creation:
    - Create a new binary feature 'is_female' based on the 'sex' column.
    - Create a new binary feature 'is_child' based on the 'age' column.
    - Create a new binary feature 'is_adult_male' based on the 'adult_male' column.
    - Create a new binary feature 'is_alone' based on the 'alone' column.
    - Create new binary features 'is_first_class', 'is_second_class', and 'is_third_class' based on the 'pclass' column.
    - Create new binary features 'embarked_C', 'embarked_Q', and 'embarked_S' based on the 'embarked' column.

- Prediction logic:
    - Initialize a variable 'y' to 0.
    - Add or subtract weights to 'y' based on the created binary features.
    - Apply the logistic function (sigmoid) to 'y' to get the final prediction.
    - Ap